In [ ]:
import simglucose
from simglucose.simulation.scenario import CustomScenario
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation
from collections import namedtuple, deque
import numpy as np
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
# Filter out deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

TODO: In testing the paper does a --> B. Step 2: Personalization of DQN-Learning Models on Patient-Specific Data.


#### Classes and Fuctions

In [ ]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
def paper_reward_function(BG_last_hour):
    G = BG_last_hour[-1]
    if G >= 70 and G <= 180:
        return 0.5
    if G > 180 and G <= 200:
        return -0.9
    if G > 200 and G <= 250:
        return -1.2
    if G > 250 and G <= 350:
        return -1.5
    if G > 30 and G < 70:
        return -1.8
    else:
        return -2

In [ ]:
def create_env(name):

    env = gym.make(name)

    env = FlattenObservation(env)

    return env

In [ ]:
class DQNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQNNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
class DDQNAgent:
    def __init__(
        self,
        input_size,
        output_size,
        gamma=0.95,
        learning_rate=0.001,
        buffer_size=800,
        batch_size=32,
    ):
        self.gamma = gamma
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Epsilon-greedy exploration
        self.epsilon_start = 1.0
        self.epsilon_end = 0.1
        self.epsilon_decay_episodes = 30

        self.epsilon = self.epsilon_start

        # Q-networks
        self.q_network = DQNNetwork(input_size, output_size).to(self.device)
        self.target_q_network = DQNNetwork(input_size, output_size).to(self.device)
        self.target_q_network.load_state_dict(self.q_network.state_dict())

        # Optimizer
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=learning_rate)

        # Experience replay buffer
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.replay_buffer = deque(maxlen=buffer_size)

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            action = env.action_space.sample()
        else:
            # Exploit: choose the action with the highest Q-value
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            action = q_values.argmax().item()

        return action

    # Update epsilon during training
    def update_epsilon(self, episode):
        # Linear decay from epsilon_start to epsilon_end over epsilon_decay_episodes
        self.epsilon = max(self.epsilon_end, self.epsilon_start - (episode / self.epsilon_decay_episodes))

    def store_transition(self, state, action, reward, next_state, done):
        transition = (state, action, reward, next_state, done)
        self.replay_buffer.append(transition)

    def sample_batch(self):
        batch = random.sample(
            self.replay_buffer, min(len(self.replay_buffer), self.batch_size)
        )
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to tensors and handle dimensions
        states = torch.stack([torch.FloatTensor(state) for state in states])
        actions = torch.LongTensor(actions).unsqueeze(1)
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.stack(
            [torch.FloatTensor(next_state) for next_state in next_states]
        )
        dones = torch.FloatTensor(dones).unsqueeze(1)

        return states, actions, rewards, next_states, dones

    def update_q_network(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        states, actions, rewards, next_states, dones = self.sample_batch()
        # put all to device
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)

        # Compute Q-values
        q_values = self.q_network(states).gather(1, actions)

        # Compute target Q-values using the target network
        target_q_values = (
            rewards
            + (1 - dones)
            * self.gamma
            * self.target_q_network(next_states).max(1)[0].detach().unsqueeze(1)
        )
     
        # Compute the Huber loss
        loss = F.smooth_l1_loss(q_values, target_q_values)

        # Update the Q-network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_q_network(self):
        # Update the target network by copying the Q-network parameters
        self.target_q_network.load_state_dict(self.q_network.state_dict())

    def train_step(self, state, action, reward, next_state, done):
        # Store the transition in the replay buffer
        self.store_transition(state, action, reward, next_state, done)

        # Update the Q-network
        self.update_q_network()

        # Update the target Q-network periodically
        if len(self.replay_buffer) % 50 == 0:
            self.update_target_q_network()

In [ ]:
class FlattenAction(gym.ActionWrapper):
    """Action wrapper that flattens the action."""

    def __init__(self, env):
        super(FlattenAction, self).__init__(env)
        self.action_space = gym.spaces.utils.flatten_space(self.env.action_space)

    def action(self, action):
        return gym.spaces.utils.unflatten(self.env.action_space, action)

    def reverse_action(self, action):
        return gym.spaces.utils.flatten(self.env.action_space, action)

In [ ]:
def evaluate_policy(policy, env_name, seed, eval_episodes=10, max_timesteps=1000):
    eval_env = create_env(env_name)
    eval_env.unwrapped.seed(seed + 100)

    avg_reward = 0.0

    for _ in range(eval_episodes):
        state, info = eval_env.reset()
        done = False

        for _ in range(max_timesteps):
            action = policy.select_action(state)
            state, reward, done, _, info = eval_env.step(action)

            avg_reward += reward

            if done:
                break

    avg_reward /= eval_episodes
    return avg_reward


#### Training

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done', 'episode'))

In [ ]:
gym.envs.register(
    id="simglucose-bolus",
    entry_point="simglucose.envs:T1DSimEnvBolus",
    kwargs={
        "patient_name": ["adolescent#001", 
                         "adolescent#002", 
                         "adolescent#003", 
                         "adolescent#004", 
                         "adolescent#005", 
                         "adolescent#006", 
                         "adolescent#007", 
                         "adolescent#008", 
                         "adolescent#009", 
                         "adolescent#010"],
        "reward_fun": paper_reward_function,
        "history_length": 1,
        "enable_meal": True,
    },
)

In [ ]:
# create train env
env = create_env('simglucose-bolus')

In [ ]:
# DDQ Agent setup
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print('state_size', state_size)
print('action_size', action_size)
agent = DDQNAgent(input_size=state_size, output_size=action_size)

In [ ]:
replay_memory_size = 800
memory = ReplayMemory(replay_memory_size)

In [ ]:
num_episodes = 100

for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0
    done = False

    while not done:  # End the episode if the environment signals that it's done
        action = agent.select_action(state)

        # Take the selected action in the environment
        next_state, reward, done, _, info = env.step(action)

        # Store the transition and perform a training step
        agent.train_step(state, action, reward, next_state, done)

        memory.push(state, action, next_state, reward, done, episode + 1)

        state = next_state

        # Accumulate the total reward
        total_reward += reward

    # Update epsilon
    agent.update_epsilon(episode)

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

In [ ]:
#state
#observation [GCM, CHO, Insulin]

In [ ]:
# visualize results from the memory
plt.figure(figsize=(10, 10))
plt.subplot(3, 2, 1)
gcm_values = [transition.state[0] for transition in memory.memory]

plt.plot(gcm_values)
plt.xlabel("Episode")
plt.ylabel("GCM")

plt.subplot(3, 2, 2)
rewards = [transition.reward for transition in memory.memory]

plt.plot(rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")

plt.subplot(3, 2, 3)
insulin_values = [transition.state[2] for transition in memory.memory]

plt.plot(insulin_values)
plt.xlabel("Episode")
plt.ylabel("Insulin")

plt.subplot(3, 2, 4)
cho_values = [transition.state[1] for transition in memory.memory]

plt.plot(cho_values)
plt.xlabel("Episode")
plt.ylabel("CHO")

plt.subplot(3, 2, 5)
# visualize the actions taken
actions_idx = [transition.action for transition in memory.memory]

actions_percentage = np.array([0.75, 0.8, 0.9, 1, 1.1, 1.2, 1.25]) - 1
actions = actions_percentage[actions_idx]

plt.plot(actions)
plt.xlabel("Episode")
plt.ylabel("Action (%)")

plt.tight_layout()
plt.show()
#'state', 'action', 'next_state', 'reward', 'done', 'episode'

In [ ]:
avg_reward =evaluate_policy(agent, "simglucose-bolus", 0)

In [ ]:
avg_reward	

Evaluation Metrics: 
- Section V. A:  we de- rived three metrics related to the the percentage of time spent within the different glycemic ranges, that is the normoglycemic range (TIR), i.e., 70 ≤ CGM ≤ 180 mg/dL, below this range (TBR), i.e., CGM < 70 mg/dL, and above this range (TAR), i.e., CGM > 180 mg/dL. 
- Figure 4